# text experiment


In [1]:
from src.CipherUtils import CipherGenerator
from src.CipherUtils import TextEncoder
import numpy as np

In [2]:
hidden_sequence = "people of western europe a landing was made this morning on the coast of france by troops kangaroo jokes quasi vile xilophone zenit "
hidden_sequence = " in germany it seems to be pretty much automatic pretty much all the time in france and spain it all depends presumably on social subtleties that you have to be french or spanish to understand in italy why would you even bother when and how much to tip is a question that has been vexing visitors to europe for as long as people have been travelling around the continent outside their own country it seems even europeans don t know the answer according to new polling by yougov in six eu countries britain and the us where as most visitors know but may be reluctant to acknowledge gratuities may make up more than half your waitperson s income europeans are deeply divided on tipping in restaurants for example of respondents in germany told the pollster they typically tipped almost the same as the us in the uk where an optional service charge of about is usually included said they left a gratuity the figure in spain where service is often included in restaurant bills but diners can leave optional tips was while in france where every price on a restaurant menu already includes for service of people said they generally tipped on top even in sweden where tips are generally not expected the figure was but only of italians said they would typically leave a gratuity after a meal out with a rather greater proportion admitting they never left a cent a startling of respondents in the us however and of germans by far the most in europe confessed they would tip sometimes or often even if the service was terrible indicating that for some tipping is not about quality of service at all the findings of the survey will come as a surprise in germany a country that does not generally think of itself as a nation of happy distributors"
# hidden_sequence = " hello banana xilophone key queue zebra cock pussy tits dandy fart though jolly world mum "


cipher_generator = CipherGenerator()
cipher = cipher_generator.generate_cipher()
encoder = TextEncoder()
observed_sequence = encoder.encode_text(hidden_sequence, cipher=cipher)


print(observed_sequence)

 vm zeqgcmt vs reegr su ie pqesst gjoa cjsugcsvo pqesst gjoa cww sae svge vm nqcmoe cmy rpcvm vs cww yepemyr pqerjgciwt um ruovcw rjiswesver sacs tuj acfe su ie nqemoa uq rpcmvra su jmyeqrscmy vm vscwt lat lujwy tuj efem iusaeq laem cmy aul gjoa su svp vr c bjersvum sacs acr ieem fekvmz fvrvsuqr su ejqupe nuq cr wumz cr peupwe acfe ieem sqcfewwvmz cqujmy sae oumsvmems ujsrvye saevq ulm oujmsqt vs reegr efem ejqupecmr yum s xmul sae cmrleq coouqyvmz su mel puwwvmz it tujzuf vm rvk ej oujmsqver iqvscvm cmy sae jr laeqe cr gurs fvrvsuqr xmul ijs gct ie qewjoscms su coxmulweyze zqcsjvsver gct gcxe jp guqe sacm acwn tujq lcvspeqrum r vmouge ejqupecmr cqe yeepwt yvfvyey um svppvmz vm qerscjqcmsr nuq ekcgpwe un qerpumyemsr vm zeqgcmt suwy sae puwwrseq saet stpvocwwt svppey cwgurs sae rcge cr sae jr vm sae jx laeqe cm upsvumcw reqfvoe oacqze un ciujs vr jrjcwwt vmowjyey rcvy saet wens c zqcsjvst sae nvzjqe vm rpcvm laeqe reqfvoe vr unsem vmowjyey vm qerscjqcms ivwwr ijs yvmeqr ocm wecfe upsvum

In [3]:
from prove.src.Probability import ProbabilityMatrix

with open("texts/moby_dick.txt", "r") as file:
    text = file.read()

alphabet = list("abcdefghijklmnopqrstuvwxyz ")


pm = ProbabilityMatrix(text=text, alphabet=alphabet)
unknown_chars = pm.unknown_chars()
pm.preprocess_text(unknown_chars=unknown_chars)

# compute probabilities
pm.compute_matrix_spaces()

In [4]:
def map_alphabet_to_numbers():
    alphabet = "abcdefghijklmnopqrstuvwxyz "
    mapping = {char: i for i, char in enumerate(alphabet)}
    return mapping

In [5]:
def translate_string_to_numbers(text, mapping):
    numbers = [mapping[char] for char in text]
    return numbers

In [6]:
hidden_ = translate_string_to_numbers(
    hidden_sequence, mapping=map_alphabet_to_numbers()
)
observed_ = translate_string_to_numbers(
    observed_sequence, mapping=map_alphabet_to_numbers()
)

In [7]:
transition_probs = pm.matrix

In [8]:
from src.HMM_functions import Baum_Welch

emission = Baum_Welch(
    A=pm.matrix,
    B_start=np.zeros((27, 27)) + 1 / 27,
    pi=pm.matrix[-1, :],
    observed=observed_,
    maxIter=500,
)

In [9]:
emission

array([[0.00000000e+000, 0.00000000e+000, 8.47353395e-001,
        0.00000000e+000, 2.86136487e-048, 2.10308461e-149,
        1.16126272e-183, 0.00000000e+000, 5.01916995e-180,
        8.53951149e-002, 5.36292164e-208, 0.00000000e+000,
        0.00000000e+000, 2.96273522e-142, 0.00000000e+000,
        5.07638522e-086, 4.96554137e-205, 8.49785628e-230,
        1.12619024e-267, 0.00000000e+000, 5.65080795e-002,
        2.61163564e-010, 4.44670180e-279, 1.07434105e-002,
        1.40900637e-263, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 5.33915574e-132, 4.42090991e-232,
        0.00000000e+000, 4.28720476e-002, 1.56659375e-140,
        4.38655532e-034, 0.00000000e+000, 8.94196950e-001,
        0.00000000e+000, 0.00000000e+000, 1.25985045e-276,
        7.78239585e-085, 1.03956982e-216, 4.59098054e-194,
        3.13958422e-204, 1.29247140e-245, 1.24509407e-140,
        1.76393047e-285, 6.11269912e-002, 0.00000000e+000,
        0.00000000e+000, 7.19181010e-187, 8.79112359e-2

In [10]:
emission.argmax(axis=1)

array([ 2,  8, 14, 24,  4, 13, 25,  0, 21, 19,  5, 22,  6, 12, 20, 25,  4,
       16, 17, 18,  9,  5, 11, 10, 19, 23, 26])

In [11]:
import string


def solve_mapping_problem(L):
    alphabet = string.ascii_lowercase + " "
    mapping = {}
    for i, num in enumerate(L):
        mapping[alphabet[i]] = alphabet[num]
    return mapping


mapping = solve_mapping_problem(emission.argmax(axis=1))

In [12]:
mapping

{'a': 'c',
 'b': 'i',
 'c': 'o',
 'd': 'y',
 'e': 'e',
 'f': 'n',
 'g': 'z',
 'h': 'a',
 'i': 'v',
 'j': 't',
 'k': 'f',
 'l': 'w',
 'm': 'g',
 'n': 'm',
 'o': 'u',
 'p': 'z',
 'q': 'e',
 'r': 'q',
 's': 'r',
 't': 's',
 'u': 'j',
 'v': 'f',
 'w': 'l',
 'x': 'k',
 'y': 't',
 'z': 'x',
 ' ': ' '}

In [13]:
def transform_string(string, mapping):
    transformed_string = ""
    for char in string:
        if char in mapping:
            transformed_string += mapping[char]
        else:
            transformed_string += char
    return transformed_string


def invert_mapping(mapping):
    inverted_mapping = {value: key for key, value in mapping.items()}
    return inverted_mapping


transform_string(observed_sequence, invert_mapping(mapping))

' in pqrmany it sqqms to bq prqtty much automatic prqtty much all thq timq in francq and spain it all dqpqnds prqsumably on social subtlqtiqs that you havq to bq frqnch or spanish to undqrstand in italy why would you qvqn bothqr whqn and how much to tip is a buqstion that has bqqn vqxinp visitors to quropq for as lonp as pqoplq havq bqqn travqllinp around thq continqnt outsidq thqir own country it sqqms qvqn quropqans don t znow thq answqr accordinp to nqw pollinp by youpov in six qu countriqs britain and thq us whqrq as most visitors znow but may bq rqluctant to acznowlqdpq pratuitiqs may mazq up morq than half your waitpqrson s incomq quropqans arq dqqply dividqd on tippinp in rqstaurants for qxamplq of rqspondqnts in pqrmany told thq pollstqr thqy typically tippqd almost thq samq as thq us in thq uz whqrq an optional sqrvicq charpq of about is usually includqd said thqy lqft a pratuity thq fipurq in spain whqrq sqrvicq is oftqn includqd in rqstaurant bills but dinqrs can lqavq optio